# Overview

This Jupyter Notebook takes in data from a Google Sheet that contains stop change details and their associated high level categories and outputs a JSON file for each line to be used in the MyBus tool.

The output file is used by the MyBus tool's results page and contains the Stop-level changes that are displayed there.

Run all cells to output files in the format `#-changes.json`

As of 5/28/21, this should output data for 141 lines.

In [13]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

DATA_INPUT_PATH = '../data/input/'
DATA_OUTPUT_PATH = '../data/stop-changes/'
SCRATCH_PATH = 'scratch/'
LINES_PATH = '../data/lines.json'
GOOGLE_SHEET_LINK = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRg9oVntzWoLImtA3AFGTQYZoTa7JPMYG1TqjV__zIxpvaBhHG_ZWPyg_jaKt7TSVke2ZH3nEmGdzi3/pub?gid=1103678554&single=true&output=csv"

## Load GTFS Data

Use `lines.json`, which was generated from the `line-dropdown` notebook.

Extract `stops.zip` to the scratch folder.

In [14]:
lines = pd.read_json(LINES_PATH)
lines_array = lines.loc[:, 'route_number'].values
lines.head()

,route_id,route_short_name,route_number
0,2,2,2
1,4,4,4
2,10,10,10
3,14,14,14
4,16,16,16


### Extract Zipped Stops

Since the stops.txt file is huge, we must zip it up and put it into the stops.zip so that we avoid checking in a big file into GitHub.

In [15]:
with ZipFile('../data/input/stops.zip', 'r') as zf:
    zf.extractall(SCRATCH_PATH)

STOPS_PATH = SCRATCH_PATH + 'stops.txt'

In [16]:
stops = pd.read_csv(STOPS_PATH,
    usecols=['stop_id','stop_name'],
    dtype={'stop_id':'int','stop_name':'string'})

stops.head()

,stop_id,stop_name
0,1,Paramount / Slauson
1,3,Jefferson / 10th
2,6,120th / Augustus F Hawkins
3,7,120th / Martin Luther King Hospital
4,12,15054 Sherman Way


## Load Stop Changes Data

Data was compiled from spreadsheets and slides provided by Service Planners. Pull the data directly from the Google Sheet.

In [17]:
# stop_changes = pd.read_csv(DATA_INPUT_PATH + 'stop_changes - ALL.csv',
#     usecols={'line', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})
stop_changes = pd.read_csv(GOOGLE_SHEET_LINK, usecols={"line","direction","CHANGE","stop_id","ST_DIR","ON_STREET","AT_BET","AT_STREET","BETWEEN_ST","NEAR_FAR","stop_added","service_canceled","service_changed","service_replaced","stop_canceled","stop_relocated","route_changed","owl_service_added"})

stop_changes.head()

,line,direction,CHANGE,stop_id,ST_DIR,ON_STREET,AT_BET,AT_STREET,BETWEEN_ST,NEAR_FAR,stop_added,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_added
0,2,eastbound,DELETE,8030,E,SUNSET,A,BONNIE BRAE,NaN,N,False,False,True,False,False,False,False,False
1,2,eastbound,DELETE,8011,E,SUNSET,A,ECHO PARK,NaN,N,False,False,True,False,False,False,False,False
2,2,eastbound,DELETE,6393,E,SUNSET,A,PORTIA,NaN,N,False,False,True,False,False,False,False,False
3,2,eastbound,DELETE,8009,E,SUNSET,A,DOUGLAS,NaN,N,False,False,True,False,False,False,False,False
4,2,eastbound,DELETE,2421,E,SUNSET,A,VIN SCULLY,NaN,N,False,False,True,False,False,False,False,False


In [18]:
# merge with stops.txt to get stop names
#stop_changes.stop_id = stop_changes.stop_id.astype('int')
stop_changes_with_names = pd.merge(stop_changes, stops, how='inner', on='stop_id')
stop_changes_with_names = stop_changes_with_names[["line","direction","CHANGE","stop_id","ST_DIR","ON_STREET","AT_BET","AT_STREET","BETWEEN_ST","NEAR_FAR","stop_added","service_canceled","service_changed","service_replaced","stop_canceled","stop_relocated","route_changed","owl_service_added"]]
stop_changes_with_names.to_csv(SCRATCH_PATH + 'stop_changes_with_names.csv')

stop_changes_with_names.head()

,line,direction,CHANGE,stop_id,ST_DIR,ON_STREET,AT_BET,AT_STREET,BETWEEN_ST,NEAR_FAR,stop_added,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_added
0,2,eastbound,DELETE,8030,E,SUNSET,A,BONNIE BRAE,NaN,N,False,False,True,False,False,False,False,False
1,2,eastbound,DELETE,8011,E,SUNSET,A,ECHO PARK,NaN,N,False,False,True,False,False,False,False,False
2,2,eastbound,DELETE,6393,E,SUNSET,A,PORTIA,NaN,N,False,False,True,False,False,False,False,False
3,2,eastbound,DELETE,8009,E,SUNSET,A,DOUGLAS,NaN,N,False,False,True,False,False,False,False,False
4,2,eastbound,DELETE,2421,E,SUNSET,A,VIN SCULLY,NaN,N,False,False,True,False,False,False,False,False


## Analyze Data

### Stops in `stop_changes` not in GTFS

Outputs results to file `stops-not-in-gtfs.json` in the `scratch` folder.

In [19]:
stops_not_in_gtfs = stop_changes[~stop_changes.stop_id.isin(stops.stop_id)]
stops_not_in_gtfs = stops_not_in_gtfs[['line', 'stop_id']]
stops_not_in_gtfs = stops_not_in_gtfs.sort_values(['line', 'stop_id'])

stops_not_in_gtfs.to_json(SCRATCH_PATH + 'stops-not-in-gtfs.json', orient='records')

stops_not_in_gtfs.head(20)

,line,stop_id
46,2,16318
118,4,680
139,51,140944
140,51,140944
159,53,5417
177,53,5423
347,110,5412
348,110,5414
349,110,5416
366,150,5507


### Duplicate `line` & `stop_id` combos

Outputs results to file `stops-changes-duplicates.json` in the `scratch` folder.

In [20]:
# Find duplicate line + stop_id combos

# exclude non-existent stop_ids
stop_changes_existing_stopids = stop_changes.loc[stop_changes['stop_id'] != 0]
stop_changes_existing_stopids = stop_changes_existing_stopids.groupby(['line', 'stop_id', 'direction']).size().reset_index(name="count")
stop_changes_existing_stopids = stop_changes_existing_stopids.loc[stop_changes_existing_stopids['count'] > 1]

# Output file with the stops that have duplicate rows.
filtered_combos = pd.merge(stop_changes_existing_stopids, stop_changes, how='inner', on=['line', 'stop_id', 'direction']).sort_values(by=['line', 'stop_id', 'direction'])
filtered_combos.to_csv(SCRATCH_PATH + 'stop-changes-duplicates.csv')

filtered_combos.head(20)

,line,stop_id,direction,count,CHANGE,ST_DIR,ON_STREET,AT_BET,AT_STREET,BETWEEN_ST,NEAR_FAR,stop_added,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_added
0,2,2738,eastbound,2,LINE 200 CHANGES TO LINE 2,E,KING BL,A,FIGUEROA,NaN,F,True,False,False,False,False,False,False,False
1,2,2738,eastbound,2,LINE 200 CHANGES TO LINE 2,E,KING BL,A,FIGUEROA,NaN,F,True,False,False,False,False,False,False,False
2,53,4528,northbound,2,ADD,W,119TH ST,A,WILLOWBROOK,NaN,F,True,False,False,False,False,False,False,False
3,53,4528,northbound,2,ADD,W,119TH ST,A,WILLOWBROOK,NaN,F,True,False,False,False,False,False,False,False
4,53,30007,northbound,2,ADD,S,WILLOWBROOK/ROSA PARKS STATION,A,WEST BAYS DOCK 14,NaN,N,True,False,False,False,False,False,False,False
5,53,30007,northbound,2,ADD,S,WILLOWBROOK/ROSA PARKS STATION,A,WEST BAYS DOCK 9,NaN,NaN,True,False,False,False,False,False,False,False
6,53,30007,southbound,2,ADD,S,WILLOWBROOK/ROSA PARKS STATION,A,WEST BAYS DOCK 14,NaN,N,True,False,False,False,False,False,False,False
7,53,30007,southbound,2,ADD,S,WILLOWBROOK/ROSA PARKS STATION,A,WEST BAYS DOCK 9,NaN,NaN,True,False,False,False,False,False,False,False
8,53,140758,southbound,2,ADD,N,WILMINGTON,A,118TH ST,NaN,N,True,False,False,False,False,False,False,False
9,53,140758,southbound,2,ADD,N,WILMINGTON,A,118TH ST,NaN,N,True,False,False,False,False,False,False,False


## Add Stops for Restored Service

Lines 110 and 550 have restored service.  The stops do not exist in the June 2021 GTFS data.  Reference GitHub [issue #62](https://github.com/LACMTA/mybus/issues/62).

In [21]:
restored_stops = pd.read_csv('../data/input/restored_stops.csv',
	usecols={'line', 'stop_id','direction'},
	dtype={'line':'int','stop_id':'string','direction':'string'})

restored_stops['service_canceled'] = False
restored_stops['service_changed'] = False
restored_stops['service_replaced'] = False
restored_stops['stop_canceled'] = False
restored_stops['stop_relocated'] = False
restored_stops['route_changed'] = False
restored_stops['owl_service_canceled'] = False
restored_stops['replaced_by_micro'] = False
restored_stops['service_restored'] = True

In [22]:
stop_changes

,line,direction,CHANGE,stop_id,ST_DIR,ON_STREET,AT_BET,AT_STREET,BETWEEN_ST,NEAR_FAR,stop_added,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_added
0,2,eastbound,DELETE,8030,E,SUNSET,A,BONNIE BRAE,NaN,N,False,False,True,False,False,False,False,False
1,2,eastbound,DELETE,8011,E,SUNSET,A,ECHO PARK,NaN,N,False,False,True,False,False,False,False,False
2,2,eastbound,DELETE,6393,E,SUNSET,A,PORTIA,NaN,N,False,False,True,False,False,False,False,False
3,2,eastbound,DELETE,8009,E,SUNSET,A,DOUGLAS,NaN,N,False,False,True,False,False,False,False,False
4,2,eastbound,DELETE,2421,E,SUNSET,A,VIN SCULLY,NaN,N,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,761,southbound,DELETE,8263,S,WESTWOOD,A,KINROSS,NaN,N,False,False,True,False,True,False,False,False
811,761,southbound,DELETE,16556,S,WESTWOOD,A,WILSHIRE,NaN,N,False,False,True,False,True,False,False,False
812,761,southbound,DELETE,3565,W,WILSHIRE,A,VETERAN,NaN,F,False,False,True,False,True,False,False,False
813,761,southbound,DELETE,16771,W,WILSHIRE,A,VETERAN,NaN,N,False,False,True,False,True,False,False,False


In [23]:
stop_changes = pd.concat([stop_changes,restored_stops])
stop_changes

,line,direction,CHANGE,stop_id,ST_DIR,ON_STREET,AT_BET,AT_STREET,BETWEEN_ST,NEAR_FAR,...,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_added,owl_service_canceled,replaced_by_micro,service_restored
0,2,eastbound,DELETE,8030,E,SUNSET,A,BONNIE BRAE,NaN,N,...,False,True,False,False,False,False,False,NaN,NaN,NaN
1,2,eastbound,DELETE,8011,E,SUNSET,A,ECHO PARK,NaN,N,...,False,True,False,False,False,False,False,NaN,NaN,NaN
2,2,eastbound,DELETE,6393,E,SUNSET,A,PORTIA,NaN,N,...,False,True,False,False,False,False,False,NaN,NaN,NaN
3,2,eastbound,DELETE,8009,E,SUNSET,A,DOUGLAS,NaN,N,...,False,True,False,False,False,False,False,NaN,NaN,NaN
4,2,eastbound,DELETE,2421,E,SUNSET,A,VIN SCULLY,NaN,N,...,False,True,False,False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,550,Southbound,NaN,15616,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,NaN,False,False,True
42,550,Southbound,NaN,10994,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,NaN,False,False,True
43,550,Southbound,NaN,10853,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,NaN,False,False,True
44,550,Southbound,NaN,2324,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,NaN,False,False,True


## Output Stop Changes Data

Iterate through lines.json and output a file for each line.

In [24]:
# Loop through line numbers and output a file for each line with all data for that line
line_count = 1
stop_count = 0

for line in lines_array:
    subset = stop_changes[stop_changes.line == line]
    filename = DATA_OUTPUT_PATH + str(line) + '-changes.json'
    
    stop_count += len(subset)
    print(subset[['line', 'direction', 'stop_id']])
    subset.to_json(filename, orient='records')
    
    print('********** ' + 'Line ' + filename + ' created (' + str(len(subset)) + ')')
    print('********** ' + str(stop_count)  + ' total stops')
    print('********** ' + str(line_count)  + ' total lines')
    line_count += 1

     line  direction stop_id
0       2  eastbound    8030
1       2  eastbound    8011
2       2  eastbound    6393
3       2  eastbound    8009
4       2  eastbound    2421
..    ...        ...     ...
113     2  westbound    4654
114     2  westbound    3357
115     2  westbound    3361
116     2  westbound    3358
117     2  westbound    3359

[118 rows x 3 columns]
********** Line ../data/stop-changes/2-changes.json created (118)
********** 118 total stops
********** 1 total lines
     line  direction stop_id
118     4  eastbound     680
119     4  eastbound   12411
120     4  westbound   11900
121     4  westbound    9221
122     4  eastbound    3587
123     4  westbound     136
124     4  westbound    8097
********** Line ../data/stop-changes/4-changes.json created (7)
********** 125 total stops
********** 2 total lines
Empty DataFrame
Columns: [line, direction, stop_id]
Index: []
********** Line ../data/stop-changes/10-changes.json created (0)
********** 125 total stops
********